In [1]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim

from DataPreprocess import create_dataloader
from InitializingModule import InitModel
from TrainingAlgorithm import train

In [2]:
train_loader, val_loader = create_dataloader(datapath='data/SPECT_data.txt', batch_size=128)

In [6]:
device = torch.device('cuda')
a = InitModel(input_size=22, hidden_size=3, output_size=1, device=device, batch_norm=True)
test_model = a.init_module_multi_ReLU_AE(train_loader)
optimizer = optim.Adam(test_model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [7]:
model = train(train_loader=train_loader, val_loader=val_loader, model=test_model, epochs=100, optimizer=optimizer, criterion=criterion, device=device)

[ 1/100 ] | train_loss = 0.59976, train_acc = 0.63994, val_loss = 0.59754, val_acc = 0.64815
[ 2/100 ] | train_loss = 0.59844, train_acc = 0.65363, val_loss = 0.59612, val_acc = 0.64815
[ 3/100 ] | train_loss = 0.59713, train_acc = 0.66342, val_loss = 0.59461, val_acc = 0.66667
[ 4/100 ] | train_loss = 0.59583, train_acc = 0.66342, val_loss = 0.59309, val_acc = 0.66667
[ 5/100 ] | train_loss = 0.59453, train_acc = 0.66733, val_loss = 0.59153, val_acc = 0.66667
[ 6/100 ] | train_loss = 0.59322, train_acc = 0.67321, val_loss = 0.58997, val_acc = 0.66667
[ 7/100 ] | train_loss = 0.59188, train_acc = 0.68300, val_loss = 0.58837, val_acc = 0.66667
[ 8/100 ] | train_loss = 0.59055, train_acc = 0.68690, val_loss = 0.58677, val_acc = 0.66667
[ 9/100 ] | train_loss = 0.58921, train_acc = 0.69278, val_loss = 0.58519, val_acc = 0.68519
[ 10/100 ] | train_loss = 0.58788, train_acc = 0.70257, val_loss = 0.58360, val_acc = 0.68519
[ 11/100 ] | train_loss = 0.58654, train_acc = 0.70257, val_loss = 0.

In [8]:
def predict(model, data_loader, criterion, device):
    
    model.to(device).eval()
    mem = []
    with torch.no_grad():
        for i, data in enumerate(data_loader.dataset):
            x, y = data[0].unsqueeze(0), data[1]
            y_pred = torch.round(torch.sigmoid(model(x))).squeeze()
            loss = criterion(y, y_pred)
            
            if y_pred != y:
                mem.append((i, loss.item()))
                
    return max(mem, key=lambda mem: mem[1])[0]


def binary_acc(y_pred, y_true):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_result_sum = (y_pred_tag == y_true).sum().float()
    acc = correct_result_sum / y_true.shape[0]
    
    return acc


def evaluate(train_loader=None, model=None, criterion=None, device=None, binary=True):
    
    model.to(device).eval()
    valid_loss = []
    valid_accs = []

    for i, batch in enumerate(train_loader.dataset):
        x, y = batch[0].unsqueeze(0), batch[1].view(-1)
        
        with torch.no_grad():
            logits = model(x.to(device))
            
            if binary:
                loss = criterion(logits, y.to(device).unsqueeze(1))
                acc = binary_acc(logits, y.to(device).unsqueeze(1))
            else:
                loss = criterion(logits, y.to(device))
                acc = (logits.argmax(dim=-1) == y.to(device)).float().mean()
                
            valid_loss.append((loss.item()))
            valid_accs.append(acc)
    
    print(f'loss:{sum(valid_loss) / len(valid_loss)} | acc:{sum(valid_accs) / len(valid_accs)}')

In [9]:
mem = predict(model, train_loader, criterion, device)

In [10]:
mem

5

In [11]:
evaluate(train_loader, model, criterion, device)

loss:0.4477453765594903 | acc:0.8591549396514893


In [12]:
model.add_neuron(train_loader, mem)

In [13]:
evaluate(train_loader, model, criterion, device)

loss:0.4742723027585258 | acc:0.7840375900268555


In [14]:
num = predict(model, train_loader, criterion, device)

In [15]:
num

17

In [20]:
sum([p.numel() for p in model.layer_out.parameters()])

5

4

In [16]:
evaluate(train_loader, model, criterion, device)

loss:0.6724752671841724 | acc:0.6009389758110046


In [17]:
train_loader.dataset[0]

(tensor([-0.9144, -0.5755, -0.7678, -0.6481, -0.8149, -0.5538, -0.6190, -0.8637,
         -0.6922, -0.7756, -0.5755, -0.6262, -0.9586, -0.6774, -0.4808, -0.6701,
          2.4230, -0.3563,  1.6543, -0.6997, -0.7146, -0.7990], device='cuda:0'),
 tensor(1., device='cuda:0'))

In [17]:
model.layer_1.bias

Parameter containing:
tensor([  0.3140,  -0.0873,   0.1117, -21.0000], device='cuda:0',
       requires_grad=True)

tensor(1.2633)

In [8]:
a = torch.randn([1])

In [9]:
b = torch.randn([3])
b = b.view(1, 3)

In [10]:
b.shape

torch.Size([1, 3])

In [11]:
torch.cat((b, a.unsqueeze(1)), dim=1)

tensor([[-1.5384,  1.9720,  0.2452, -0.2344]])

In [12]:
a.unsqueeze(1).shape

torch.Size([1, 1])

In [13]:
b

tensor([[-1.5384,  1.9720,  0.2452]])